In [23]:
import pandas as pd 
from sklearn import svm
import  numpy as np
from sklearn.externals import joblib
import os.path

In [24]:
#Only US so far. 
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [25]:
df = pd.read_csv('us-5K_TRAINING.csv',sep=',',encoding='utf-8',header=None)

In [26]:
df.head(n=5)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,James,Butt,"Benton, John B Jr",6649 N Blue Gum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com,http://www.bentonjohnbjr.com,True
1,Jams,Buttj,"BentonI, JohnZB Jr",6649 N Blue GPm St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com,http://www.bentonjohnbjr.com,False
2,Jams,Buttq,"Bentxon, Jocn B Jr",6649 N Blue GPm St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com,http://www.bentonjohnbjr.com,False
3,Jmes,Blutt,BentAnR John B Jr,6649 N BlueNGum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com,http://www.bentonjohnbjr.com,False
4,Josephine,Darakjy,"Chanay, Jeffrey A Esq",4 B Blue Ridge Blvd,Brighton,Livingston,MI,48116,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org,http://www.chanayjeffreyaesq.com,True


In [27]:
names= df[0]
last = df[1]
add  = df[3]
zipp = df[7]
sts = df[6]
print ("Max lenght name: ",np.array([len(i) for i in names]).max())
print ("Max lenght lastname: ",np.array([len(i) for i in last]).max())
print ("Max lenght address: ",np.array([len(i) for i in add]).max())

Max lenght name:  13
Max lenght lastname:  15
Max lenght address:  35


In [28]:
zipcodes=list(set(zipp))
print("Number zip codes:", len(zipcodes))


Number zip codes: 2111


In [29]:
names_matrix=np.zeros((len(df),30)) 
last_matrix=np.zeros((len(df),30))
add_matrix=np.zeros((len(df),64))


In [30]:
x='6649 N Blue Gum St'
print(len(x))
print(np.array([ord(c) for c in x]))

18
[ 54  54  52  57  32  78  32  66 108 117 101  32  71 117 109  32  83 116]


In [31]:
x='John'
print(len(x))
print(np.array([ord(c) for c in x]))

4
[ 74 111 104 110]


In [32]:
name_array=[[ord(c) for c in i] for i in names]
last_array=[[ord(c) for c in i] for i in last]
add_array=[[ord(c) for c in i] for i in add]

In [33]:
def fill_matrix(matrix,ls,samples):
    for i in range(samples):
        for j in range (len(ls[i])):
            matrix[i][j]=ls[i][j]
    return matrix

In [34]:
names_matrix = fill_matrix(names_matrix,name_array,len(df))
last_matrix = fill_matrix(last_matrix,last_array,len(df))
add_matrix = fill_matrix(add_matrix,add_array,len(df))

In [35]:
states_array = np.array([states.index(i) for i in sts ]).reshape(-1,1)
zip_array = np.array([zipcodes.index(i) for i in zipp ]).reshape(-1,1)

In [36]:
X_train = np.concatenate((names_matrix, last_matrix,add_matrix,states_array,zip_array), axis=1)


In [37]:
df_features = pd.DataFrame(X_train)
df_features.head()

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
0,74.0,97.0,109.0,101.0,115.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,1249.0
1,74.0,97.0,109.0,115.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,1249.0
2,74.0,97.0,109.0,115.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,1249.0
3,74.0,109.0,101.0,115.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,1249.0
4,74.0,111.0,115.0,101.0,112.0,104.0,105.0,110.0,101.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,1844.0


In [38]:
df_features.shape

(20000, 126)

In [39]:
def predict_item(item,model):
    name=[ord(c) for c in item[0]]
    last = [ord(c) for c in item[1]]
    add = [ord(c) for c in item[2]]
    
    names_matrix=np.zeros((1,30))
    last_matrix=np.zeros((1,30))
    add_matrix=np.zeros((1,64))
    
    names_matrix = fill_matrix(names_matrix,name_array,1)
    last_matrix = fill_matrix(last_matrix,last_array,1)
    add_matrix = fill_matrix(add_matrix,add_array,1)
    
    
    states_array = np.array([states.index(item[3])]).reshape(-1,1)
    zip_array = np.array([zipcodes.index(item[4])]).reshape(-1,1)
    
    test = np.concatenate((names_matrix, last_matrix,add_matrix,states_array,zip_array), axis=1)
    y_pred=model.predict(test)
    if y_pred==1:
        print("Denied! {0} {1} is on the black-list".format(item[0],item[1]))
    else:
        print("Good News! {0} {1} is on the white-list".format(item[0],item[1]))
    return(y_pred)

### OneClass SVM Model RBF Kernel

In [40]:

rbf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

In [41]:
rbf

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [42]:
filename = 'one_svm_rbf.sav'
if os.path.exists(filename):
    rbf = joblib.load(filename)
else:
    
    rbf.fit(X_train)
    joblib.dump(rbf, filename)

In [43]:
sample= ["Jams","Butdt","6649 N Blsue Gum St", "LA", 70116]
ypred=predict_item(sample,rbf)

Denied! Jams Butdt is on the black-list


In [ ]:
sample= ["Wayner","Barrios","5000 Forbes ave", "PA", 12343]
ypred=predict_item(sample,rbf)

In [45]:
sample= ["Seymour","Okelley","33430 Hylan Blvd","CA",95404]
ypred=predict_item(sample,rbf)

Good News! Seymour Okelley is on the white-list


In [46]:
y_pred_train = rbf.predict(X_train)

In [147]:
n_error_train = y_pred_train[y_pred_train == -1].size
print("Training error (%):" ,(n_error_train/len(df))*100)

Training error (%): 55.435


### OneClass SVM Model linear Kernel

In [220]:
linear = svm.OneClassSVM(nu=0.01, kernel="linear", gamma=0.1)

In [221]:
linear

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='linear',
      max_iter=-1, nu=0.01, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [16]:
filename = 'one_svm_linear.sav'
if os.path.exists(filename):
    linear = joblib.load(filename)
else: 
    linear.fit(X_train)
    joblib.dump(linear, filename)

In [28]:
sample= ["Jaaams","Butttt","6649 N Bsdhe Gum St", "LA",70116]
ypred=predict_item(sample,linear)

Denied! Jaaams Butttt is on the black-list


In [17]:
sample= ["ames","Buttsdt","6649 N Blueuae Gum St", "LA",70116]
ypred=predict_item(sample,linear)

Denied! ames Buttsdt is on the black-list


In [226]:
sample= ["Jorge","Quiroga-Sanchez","2395 NW 81st Terrace", "FL", 33147]
ypred=predict_item(sample,linear)

Good News! Jorge Quiroga-Sanchez is on the white-list


In [227]:
sample= ["Joe","Quiroga","2395 NW 81st Terrace", "FL", 33147]
ypred=predict_item(sample,linear)

Good News! Joe Quiroga is on the white-list


In [228]:
sample= ["Shakira","Mebarak","530 Krueger St", "LA", 90232]
ypred=predict_item(sample,linear)



Good News! Shakira Mebarak is on the white-list


In [229]:
sample= ["Seymour","Okelley","33430 Hylan Blvd","CA",95404]
ypred=predict_item(sample,linear)

Denied! Seymour Okelley is on the black-list


In [230]:
sample= ["Seyssmour","Okelddley","33430 Hylan Blvd","LA",95404]
ypred=predict_item(sample,linear)

Denied! Seyssmour Okelddley is on the black-list


In [231]:
sample= ["Seyssur","Okellesdsy","33430 Hylan Blvd","WA",95404]
ypred=predict_item(sample,linear)

Denied! Seyssur Okellesdsy is on the black-list


In [29]:
sample= ["Yolanda","Svay","80 School St","AZ",85257]
ypred=predict_item(sample,linear)


Denied! Yolanda Svay is on the black-list


In [31]:
sample= ["Wayner","Barrios","80 School St","AZ",85257]
ypred=predict_item(sample,linear)


Denied! Wayner Barrios is on the black-list


In [34]:
y_pred_train = linear.predict(X_train)

In [235]:
n_error_train = y_pred_train[y_pred_train == -1].size
print("Training error (%):" ,(n_error_train/len(df))*100)

Training error (%): 10.005


In [35]:
y_pred_train[y_pred_train == -1].size

2001